In [53]:
from glob import glob
from PIL import Image
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
from geopandas import GeoDataFrame
import json
import os
from shutil import copyfile

In [19]:
def _add_trailing_slash(path):
    if path[-1] != '/':
        path += '/'
    return path


def _create_dir(output_dir):
    # If the output folder doesn't exist, create it
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
        

        
        
def csv_to_gdf(csv_loc):
    '''
    import manually created areas of interest csv

    output is an in-memory geo dataframe with one polygon AOI per row to be utilized for cropping master raster

    '''
#     csv_loc = self.csv_loc
    df = pd.read_csv(csv_loc)
    df_labels = df[["polygon","Labels combined"]]

    #create geometry column for polygons
    polygons = []
    for polygon in df_labels["polygon"]:
        polygon_temp = []
        for coordinates in json.loads(polygon)["coordinates"]:
            for coordinate in coordinates:
                polygon_temp.append(tuple(coordinate))
            polygons.append(Polygon(polygon_temp))

    gdf_series = gpd.GeoSeries(polygons)
    gdf = gpd.GeoDataFrame(gdf_series,geometry=0)
    gdf["geometry"] = gdf[0]
    gdf = gdf.drop(columns=[0])

    # add Labels column 
    gdf["Labels"] = [s.strip().split(", ") for s in df_labels["Labels combined"]]

    return gdf
            
                    
# def tif_to_jpg(in_dir, out_dir):
#     in_dir_base = _add_trailing_slash(in_dir)

#     out_dir_base = _add_trailing_slash(out_dir)

#     # If the output parent folder doesn't exist, create it

#     _create_dir(out_dir)

#     # List containing respective label directories

#     in_dir_list = glob(in_dir_base + "*/")

#     for in_dir_child in in_dir_list:

#         print('in_dir_child:', in_dir_child)
#         if self.windows:
#             label = in_dir_child.split('\\')[-2]
#         else:
#             label = in_dir_child.split("/")[-2]

#         # If output child folder doesn't exist, create

#         out_dir_child = out_dir_base + label

#         out_dir_child = _add_trailing_slash(out_dir_child)

#         _create_dir(out_dir_child)


#         # Export Polygons from TIF to  JPEG

#         tif_list = glob(in_dir_child + "*.tif",recursive=True)

#         for tif_path in tif_list:
#             print('tif_path:', tif_path)
#             if self.windows:
#                 base_filename = tif_path.split('\\')[-1].split('.')[0]
#             else:
#                 base_filename = tif_path.split("/")[-1].split(".")[0]
#             im = Image.open(tif_path)
#             im.thumbnail(im.size)
#             im.save(out_dir_child + base_filename + ".jpg", "JPEG", quality=100)

In [71]:
def export_aoi_polygon_rasters(gdf, labelled_raster_path, output_dir):
    
    
    output_parent_dir = _add_trailing_slash(output_dir) 
    # create parent output directory if it doesn't exist
    _create_dir(output_dir)
    raster_list = glob(labelled_raster_path + "/*")
    
    
    for label_tif in raster_list:
        
        index = int(label_tif.split("_")[18].split("/")[1]) - 1 
        print("tile_id:",index + 1)
        for labels in gdf.loc[[index],"Labels"]:
            for label in labels:
                # output directory per label
                output_label_dir = output_parent_dir + label
                output_label_dir = _add_trailing_slash(output_label_dir) 

                # create output directory if it doesn't exist
                _create_dir(output_label_dir)


                # output file path
                outpath = output_label_dir + str(index+1) + "_" + label + '.tif'
                print("copying",index+1,label)
                if os.path.exists(outpath):
                    outpath = output_label_dir + str(index+1) + "_" + label + "_2" '.tif'
                    copyfile(label_tif,outpath)
                        
                else:
                    copyfile(label_tif,outpath)

In [62]:
help(copyfile)

Help on function copyfile in module shutil:

copyfile(src, dst, *, follow_symlinks=True)
    Copy data from src to dst.
    
    If follow_symlinks is not set and src is a symbolic link, a new
    symlink will be created instead of copying the file it points to.



In [31]:
csv_loc = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"

gdf = csv_to_gdf(csv_loc)

In [72]:
input_raster_labels = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/GEE/tiles_v3/polygon_exports/v8_dynamic_date_range_v4_native_secondary_sort_using_area/misha_polygons_final_version"
output_dir = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v4"

export_aoi_polygon_rasters(gdf, input_raster_labels, output_dir)

tile_id: 78
copying 78 Rainforest
tile_id: 84
copying 84 ISL
copying 84 Rainforest
copying 84 Roads
copying 84 Mining
tile_id: 48
copying 48 ISL
copying 48 Rainforest
tile_id: 89
copying 89 ISL
copying 89 Rainforest
copying 89 Roads
tile_id: 88
copying 88 ISL
copying 88 Rainforest
tile_id: 82
copying 82 ISL
copying 82 Rainforest
tile_id: 12
copying 12 Water
tile_id: 75
copying 75 ISL
copying 75 Rainforest
tile_id: 76
copying 76 ISL
copying 76 Rainforest
copying 76 River
tile_id: 25
copying 25 ISL
copying 25 Rainforest
tile_id: 9
copying 9 Savannah
copying 9 Industrial agriculture
copying 9 Roads
copying 9 Shifting cultivation
copying 9 Habitation
tile_id: 10
copying 10 Rainforest
copying 10 Roads
copying 10 Industrial agriculture
copying 10 Shifting cultivation
tile_id: 56
copying 56 Roads
copying 56 Shifting cultivation
tile_id: 74
copying 74 ISL
copying 74 Rainforest
copying 74 Roads
tile_id: 60
copying 60 River
copying 60 Rainforest
tile_id: 77
copying 77 ISL
copying 77 Rainforest
t

In [ ]:
# copyfile(row["Filepath"],output_file)